In [4]:
import pandas as pd

In [3]:
df=pd.read_excel("filtered_data.xlsx")
print(df.shape)

(86261, 7)


In [35]:
columns=['Гүйлгээний огноо', 'Харилцагчийн код', 'Харилцагчийн нэр', 'Дебет - ₮', 'Кредит -₮',"Тооцооны төрөл" ]


avlaga_nasjilt=pd.read_excel("Авлага_насжилт.xlsx", usecols=columns)
print(avlaga_nasjilt.shape)
print(avlaga_nasjilt.columns)

(10328, 6)
Index(['Гүйлгээний огноо', 'Тооцооны төрөл', 'Харилцагчийн код',
       'Харилцагчийн нэр', 'Дебет - ₮', 'Кредит -₮'],
      dtype='object')


In [36]:
# Огноог datetime болгох
avlaga_nasjilt['Гүйлгээний огноо'] = pd.to_datetime(avlaga_nasjilt['Гүйлгээний огноо'], errors='coerce')
# avlaga_nasjilt['Гүйлгээний огноо'] = pd.to_datetime(avlaga_nasjilt['Гүйлгээний огноо'], format='%Y-%m-%d')
# avlaga_nasjilt['Гүйлгээний огноо'] = avlaga_nasjilt['Гүйлгээний огноо'].dt.strftime('%Y-%m-%d')
# Сар багана гаргах
avlaga_nasjilt['Сар'] = avlaga_nasjilt['Гүйлгээний огноо'].dt.to_period('M')

In [37]:
avlaga_nasjilt["Үлдэгдэл"] = avlaga_nasjilt['Дебет - ₮'] - avlaga_nasjilt['Кредит -₮']

In [38]:
pivot_sorted= avlaga_nasjilt.sort_values(['Харилцагчийн код', 'Харилцагчийн нэр', 'Тооцооны төрөл', 'Сар'])
pivot_sorted['Нийт үлдэгдэл'] = pivot_sorted.groupby(['Харилцагчийн код', 'Харилцагчийн нэр', 'Тооцооны төрөл',])['Үлдэгдэл'].cumsum()
latest_balances = pivot_sorted.groupby(['Харилцагчийн код', 'Харилцагчийн нэр', 'Тооцооны төрөл']).last().reset_index()
# pivot_table = avlaga_nasjilt.groupby(['Харилцагчийн код', 'Харилцагчийн нэр', 'Сар'])["Үлдэгдэл"].sum().reset_index()
latest_balances = latest_balances[latest_balances['Нийт үлдэгдэл'] != 0]

In [39]:
latest_balances.to_excel("latest_balances.xlsx", index=False)

In [26]:
last_c2=pivot_filtered.sort_values("Сар").groupby(['Харилцагчийн код', 'Харилцагчийн нэр']).tail(1)

In [27]:
last_c2.to_excel("pivot_filtered.xlsx", index=False)

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
df=pd.read_excel("ЕЖ.xlsx")
print(df.columns)


# pd.read_excel("ЕЖ.xlsx", usecols=['Гүйлгээний утга', 'Харилцагчийн нэр', 'Тооцооны төрөл', 'Сар', 'Нийт үлдэгдэл']).to_excel("ЕЖ.xlsx", index=False)

Index(['Дугаар', 'ДТ2', 'КТ2', 'ДТ4', 'КТ4', 'ДТ6', 'КТ6', 'сар, өдөр',
       'дугаар\n', 'Гүйлгээний утга', 'Дүн', 'Дебет', 'Кредит\n'],
      dtype='object')


In [18]:
df['Огноо'] = pd.to_datetime(df['сар, өдөр'])
# print(df["Дүн"].head(10))
debet_df = df[["Огноо", "Дебет", "Дүн"]].rename(columns={"Дебет": "Данс код"})
debet_df["Сар"] = debet_df["Огноо"].dt.to_period("M")
pivot = debet_df.groupby([ "Данс код"])["Дүн"].sum().reset_index()


In [ ]:
# Дүн баганыг тоон төрөлд хөрвүүлэх
pivot['Дүн'] = pd.to_numeric(pivot['Дүн'], errors='coerce')

# График зурна
import matplotlib.pyplot as plt
import seaborn as sns

print(pivot["Дүн"].dtype)
print(pivot["Дүн"].head(10))
print("PN тооцоо.xlsx")
# print(pivot["Сар"].dtype)
plt.figure(figsize=(14, 6))
# pivot['Сар'] = pivot['Сар'].astype(str)  # Сар баганыг string төрөлд хөрвүүлэх
pivot["Дүн"]=pivot["Дүн"].map('{:.2f}'.format)  # Дүн баганыг форматлах
print(pivot["Дүн"].head(10))
sns.lineplot(data=pivot, x='Сар', y='Дүн', hue='Данс код')

plt.title('Сар бүрийн дүн (данс тус бүрээр)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(data=monthly, x='Сар', y='Дүн')

plt.title('Сар бүрийн нийт дүн')
plt.xticks(rotation=45)
plt.ylabel('Дүн (₮)')
plt.xlabel('Сар')
plt.tight_layout()
plt.show()

In [16]:
import pandas as pd
df = pd.read_excel("Авлага_2023.03.17.xlsx")
df=df.iloc[:197]
print(df.shape)
print(df.columns)
print(df.head(10))
print(df.info())

(194, 6)
Index(['Харилцагчийн код', 'Харилцагчийн нэр', 'Эхний үлдэгдэл', 'Дебит',
       'Кредит', 'Эцсийн үлдэгдэл'],
      dtype='object')
         Харилцагчийн код Харилцагчийн нэр  Эхний үлдэгдэл    Дебит    Кредит  \
0  120101 - Дансны авлага              NaN             NaN      NaN       NaN   
1                   10003          Соёлзул        106920.0      0.0       0.0   
2                   10004          Билгүүн        675465.0      0.0       0.0   
3                   10006        Цэнд-Аюуш       -356800.0      0.0       0.0   
4                   10007         Энхцэцэг         76000.0      0.0       0.0   
5                   10009       Үүрийнтуяа         47650.0      0.0       0.0   
6                    1001        Билгүүн.Г      22846500.0      0.0       0.0   
7                   10011        Бямбацогт        460400.0  37800.0  108000.0   
8                   10020            Номин         22950.0      0.0       0.0   
9                    1004        Цогжаргал      

In [17]:
print(df.isna().sum())

Харилцагчийн код     0
Харилцагчийн нэр    19
Эхний үлдэгдэл       5
Дебит                5
Кредит               5
Эцсийн үлдэгдэл      0
dtype: int64


In [18]:
print(df["Харилцагчийн код"].unique())




['120101 - Дансны авлага' '10003' '10004' '10006' '10007' '10009' '1001'
 '10011' '10020' '1004' '1021' '1201001' '1201002' '1201007' '1201008'
 '1201009' '1201010' '120101011' '12010103' '1201012' '1201014' '1201015'
 '1201017' '1201018' '1201020' '1201026' '1201027' '1201028' '1201032'
 '1201033' '1201035' '1201036' '1201037' '1201039' '1201048' '1201068'
 '1201069' '1201075' '1201077' '1201079' '1201080' '1201081' '1201082'
 '1201083' '1201088' '1201089' '1201090' '1201091' '1201092' '1201094'
 '1201096' '1201097' '1201099' '1201106' '1201110' '1201116' '1201120'
 '1201122' '1201123' '1201124' '1201125' '1201126' '1201131' '1201132'
 '12011333' '1201144' '12011441' '1201148' '1201150' '1201154' '1201155'
 '1201160' '1201162' '1201163' '1201166' '1201200' '1201201' '1201202'
 '1201204' '1201205' '1201206' '1201444' '1401066' '2082489' '2083817'
 '20838172' '20838176' '2572265' '2811715' '2824795' '28247952' '3624714'
 '5046688' '50466881' '504668811' '504668812' '504668813' '50466882

In [19]:
unique_values = df["Харилцагчийн код"].unique()
# print(unique_values)
print("Төрлүүд:", set(type(x) for x in unique_values))

Төрлүүд: {<class 'str'>}


In [ ]:
mask = df["Харилцагчийн код"].astype(str).str.contains(r'^.{2}.*[А-Яа-я]', na=False)
letters_only = df[mask]
print(letters_only)


                      Харилцагчийн код Харилцагчийн нэр  Эхний үлдэгдэл  \
0               120101 - Дансны авлага              NaN             NaN   
146                         Дансны дүн              NaN    3.256527e+08   
147      120102 - Дансны авлага /Евро/              NaN             NaN   
162                         Дансны дүн              NaN    2.501477e+08   
164   120105 - Хувь хүнээс авах авлага              NaN             NaN   
174                         Дансны дүн              NaN    2.433135e+08   
175              120106 - Бусад авлага              NaN             NaN   
182                         Дансны дүн              NaN    1.121720e+07   
183  120601 - Ажиллагчдаас авах авлага              NaN             NaN   
192                         Дансны дүн              NaN    2.140303e+07   
193                               Нийт              NaN    8.517341e+08   

          Дебит    Кредит Эцсийн үлдэгдэл  
0           NaN       NaN             MNT  
146  590090

In [11]:
# Давхардаагүй утгууд дотроос бүх тэмдэгт нь зөвхөн цифр (0–9) байгааг шүүх
only_digits = df["Харилцагчийн код"].dropna().astype(str)
only_digits = only_digits[only_digits.str.isdigit()]

print(only_digits.unique())
print(f"Нийт: {only_digits.nunique()} ширхэг цэвэр тоон утга байна.")


['10003' '10004' '10006' '10007' '10009' '1001' '10011' '10020' '1004'
 '1021' '1201001' '1201002' '1201007' '1201008' '1201009' '1201010'
 '120101011' '12010103' '1201012' '1201014' '1201015' '1201017' '1201018'
 '1201020' '1201026' '1201027' '1201028' '1201032' '1201033' '1201035'
 '1201036' '1201037' '1201039' '1201048' '1201068' '1201069' '1201075'
 '1201077' '1201079' '1201080' '1201081' '1201082' '1201083' '1201088'
 '1201089' '1201090' '1201091' '1201092' '1201094' '1201096' '1201097'
 '1201099' '1201106' '1201110' '1201116' '1201120' '1201122' '1201123'
 '1201124' '1201125' '1201126' '1201131' '1201132' '12011333' '1201144'
 '12011441' '1201148' '1201150' '1201154' '1201155' '1201160' '1201162'
 '1201163' '1201166' '1201200' '1201201' '1201202' '1201204' '1201205'
 '1201206' '1201444' '1401066' '2082489' '2083817' '20838172' '20838176'
 '2572265' '2811715' '2824795' '28247952' '3624714' '5046688' '50466881'
 '504668811' '504668812' '504668813' '50466882' '50466885' '50466886'
 

In [14]:
mask = df["Харилцагчийн код"].astype(str).str.contains(r'[a-zA-Z]', na=False)
letters_only = df[mask]
print(letters_only)


    Харилцагчийн код Харилцагчийн нэр  Эхний үлдэгдэл  Дебит  Кредит  \
149              EUR              NaN        16132.49    0.0     0.0   
151              EUR              NaN        20688.35    0.0     0.0   
153              EUR              NaN        26213.79    0.0     0.0   
155              EUR              NaN         3679.20    0.0     0.0   
157              EUR              NaN           67.92    0.0     0.0   
159              EUR              NaN           74.96    0.0     0.0   
161              EUR              NaN         2275.84    0.0     0.0   
163              EUR              NaN        69132.55    0.0     0.0   

    Эцсийн үлдэгдэл  
149        16132.49  
151        20688.35  
153        26213.79  
155          3679.2  
157           67.92  
159           74.96  
161         2275.84  
163        69132.55  


In [44]:
import pandas as pd

# Excel файл унших
df = pd.read_excel("Авлага_2023.03.17.xlsx", skiprows=0, nrows=192)

# Бүх мөрийг тэмдэгт болгож авна
df["Харилцагчийн код"] = df["Харилцагчийн код"].astype(str)

# Блок бүрийн эхлэл (жишээ: "120101 - Дансны авлага") ба төгсгөл ("Дансны дүн") индексүүдийг хайна
# start_indices = df[df["Харилцагчийн код"].str.contains(r"^\d{6} - авлага", na=False)].index.tolist()
start_indices = df[df["Харилцагчийн код"].str.contains(r".*авлага", na=False)].index.tolist()

end_indices = df[df["Харилцагчийн код"] == "Дансны дүн"].index.tolist()

# Авсан блокуудаа нэг dataframe-д нэгтгэх
all_blocks = []

for start in start_indices:
    # Энд хамгийн ойр дараагийн 'Дансны дүн'-г олно
    end = next((i for i in end_indices if i > start), None)
    if end is None:
        end = df.shape[0]  # Хэрэв дараагийн 'Дансны дүн' олдсонгүй бол сүүлчийн мөр хүртэл авна
    block = df.loc[start+1:end].copy()  # Дансны нэрийг оруулахгүй
    block["Харьяалах данс"] = df.loc[start, "Харилцагчийн код"]  # тухайн блокын дансны нэр
    all_blocks.append(block)
        


# Бүгдийг нэг dataframe-д нэгтгэх
result_df = pd.concat(all_blocks, ignore_index=True)
result_df = result_df[result_df["Харилцагчийн код"] != "Дансны дүн"]  # "Дансны дүн" мөрийг хасна

result_df.to_excel("avlaga_c1_filder.xlsx", index=False)

# ✅ Үр дүн харуулах
print(result_df.head())


  Харилцагчийн код Харилцагчийн нэр  Эхний үлдэгдэл  Дебит  Кредит  \
0            10003          Соёлзул        106920.0    0.0     0.0   
1            10004          Билгүүн        675465.0    0.0     0.0   
2            10006        Цэнд-Аюуш       -356800.0    0.0     0.0   
3            10007         Энхцэцэг         76000.0    0.0     0.0   
4            10009       Үүрийнтуяа         47650.0    0.0     0.0   

  Эцсийн үлдэгдэл          Харьяалах данс  
0          106920  120101 - Дансны авлага  
1          675465  120101 - Дансны авлага  
2         -356800  120101 - Дансны авлага  
3           76000  120101 - Дансны авлага  
4           47650  120101 - Дансны авлага  


In [40]:
import os
from pathlib import Path
import pandas as pd


file_path = r'C:\Users\Dell\Downloads\PN тооцоо\data\бараа_тоо_ширхэгээр.xlsx'

print(file_path)
df=pd.read_excel(file_path, skiprows=4 )
movement_df = df[df["Баримтын төрөл"] == "Бараа материал хөдөлгөөн"].copy()
# Тоон талбаруудыг найдвартай тоон болгож авна
movement_df["Орлого тоо"] = pd.to_numeric(movement_df["Орлого тоо"], errors='coerce').fillna(0)
movement_df["Зарлага тоо"] = pd.to_numeric(movement_df["Зарлага тоо"], errors='coerce').fillna(0)
# Групплэх түлхүүр: № + Багц
grouped = movement_df.groupby(["№", "Багц"])

result_rows = []

for (no, batch), group in grouped:
    if group.shape[0] != 2:
        continue  # зөвхөн 2 мөртэй бүлэг (гарсан/орсон) гэж үзнэ

    row1, row2 = group.iloc[0], group.iloc[1]

    if row1["Зарлага тоо"] > 0:
        from_loc, to_loc = row1["Байршил"], row2["Байршил"]
        qty = row1["Зарлага тоо"]
    else:
        from_loc, to_loc = row2["Байршил"], row1["Байршил"]
        qty = row2["Зарлага тоо"]

    result_rows.append({
        "№": no,
        "Багц": batch,
        "Гаралтын агуулах": from_loc,
        "Хүлээн авсан агуулах": to_loc,
        "Шилжсэн тоо": qty
    })

# DataFrame болгох
movement_summary = pd.DataFrame(result_rows)
# Excel файл руу бичих
movement_df.to_excel("movement_summary.xlsx", index=False)

print(f"Results saved to {movement_df}")


C:\Users\Dell\Downloads\PN тооцоо\data\бараа_тоо_ширхэгээр.xlsx
Results saved to               №  Баримт                 Багц Баримтын огноо Харилцагчийн код  \
65          NaN     9.0  2402061628046791035     2023-03-20              NaN   
66          NaN     9.0  2402061628046791035     2023-03-20              NaN   
67          NaN     9.0  2402061628046791035     2023-03-20              NaN   
68          NaN     9.0  2402061628046791035     2023-03-20              NaN   
69          NaN     9.0  2402061628046791035     2023-03-20              NaN   
...         ...     ...                  ...            ...              ...   
38847  ХН110001  5125.0                  NaN     2025-04-15              NaN   
38848  ХН110001  5125.0                  NaN     2025-04-15              NaN   
38849  ХН110001  5125.0                  NaN     2025-04-15              NaN   
38850  ХН110001  5125.0                  NaN     2025-04-15              NaN   
38851  ХН110001  5125.0                